In [1]:
# ! pip install pycryptodome

In [4]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

In [5]:
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
from Crypto.Util import strxor
from Crypto.Util.Padding import pad, unpad
from Crypto.Util.strxor import strxor

In [7]:
import hashlib
import math

class Encryption():

    @staticmethod
    def hash_to_bytes(hash_hex):
        return bytes.fromhex(hash_hex)

    @staticmethod
    def xor_hashes(bytes1, hash2):
        bytes2 = Encryption.hash_to_bytes(hash2)

        # Perform XOR operation
        xor_result = bytes(a ^ b for a, b in zip(bytes1, bytes2))

        return xor_result

    def Cipher_Text(IV, Key, P):
        b_1 = Key + IV
        sha256 = hashlib.sha256()
        sha256.update(b_1)
        b1_hash= sha256.hexdigest()
        Cipher= []

        for i in range(math.ceil(len(P)/32)):
            Pi = P[31*i:31*(i+1)]
            Ci = Encryption.xor_hashes(Pi, b1_hash)
            print("Ci: ",Ci)

            concat = Key + Ci
            B = hashlib.sha256()
            B.update(concat)
            b1_hash= B.hexdigest()
            Cipher.append(Ci) 

        print("Cipher chunks array: ", Cipher)
        return Cipher




In [72]:
class Decryption():

    @staticmethod
    def hash_to_bytes(hash_hex):
        return bytes.fromhex(hash_hex)

    @staticmethod
    def xor_hashes(hash1, bytes2):
        bytes1 = Decryption.hash_to_bytes(hash1)
        xor_result = bytes(a ^ b for a, b in zip(bytes2, bytes1))

        return xor_result

    def Plain_Text(IV, Key, C):
        a = Key + IV
        b = hashlib.sha256()
        b.update(a)
        Hash_b= b.hexdigest()
        print("Hash_b initially: ", Hash_b)
        
        Plains = []
        concat = Key
        for i in C:
            print("Chunk: ",i)
            P = Decryption.xor_hashes(Hash_b, i)
            concat += P
            B = hashlib.sha256()
            B.update(concat)
            Hash_b= B.hexdigest()
            print("Hash_b After: ", Hash_b)
            Plains.append(P)

        return Plains


In [82]:
def hash_256(message):
    sha_256 = SHA256.new(message).digest()
    return sha_256

In [180]:
def XOR(m1_bytes, m2_bytes):
    xor_result = bytes(a ^ b for a, b in zip(m1_bytes, m2_bytes))
    return xor_result

In [187]:
pad(b"a", 32)

b'a\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f\x1f'

In [184]:
XOR(pad(b"a", b"ba"))

b'\x03\x0f'

In [190]:
IV = b'4832500747'
Key = b'4103583911'
Message= b"Firstly, the Customer sends Firstly, the Customer sends"
# Message_Chunks = Encryption.Cipher_Text(IV, Key, Message)

In [191]:
p1 = Message[:32]
p2 = Message[32: ]
print(f"{p1=}, {p2=}")

p1=b'Firstly, the Customer sends Firs', p2=b'tly, the Customer sends'


In [192]:
b1 = hash_256(Key + IV)
c1 = strxor(p1, b1)
print(f"{p1=}\n{b1=}\n{c1=}")

p1=b'Firstly, the Customer sends Firs'
b1=b'\xcc3\xcb\xbe\x19\xdaL\xde\xab;D\xdf\xe7\xe4\xe8w/\xeaB\xd6\xfe\x01*\xe2y\n\x96\xba?V\xbf\x02'
c1=b'\x8aZ\xb9\xcdm\xb65\xf2\x8bO,\xba\xc7\xa7\x9d\x04[\x85/\xb3\x8c!Y\x87\x17n\xe5\x9ay?\xcdq'


In [193]:
strxor(c1, b1)

b'Firstly, the Customer sends Firs'

In [195]:
p2

b'tly, the Customer sends'

In [198]:
b2 = hash_256(Key + c1)
p2_pad = pad(p2, 32)
c2 = strxor(p2_pad, b2)
print(f"{p2_pad=}\n{b2=}\n{c2=}")

p2_pad=b'tly, the Customer sends\t\t\t\t\t\t\t\t\t'
b2=b'&\xfeN\tj\x02px\n\xc6\xff\xd7|g\xc99\x86\xc0\xd9\x03\xf7w`\xa7\x92\xa9\xd3\x97\xd3\xa2~\x16'
c2=b'R\x927%Jv\x18\x1d*\x85\x8a\xa4\x08\x08\xa4\\\xf4\xe0\xaaf\x99\x13\x13\xae\x9b\xa0\xda\x9e\xda\xabw\x1f'


In [199]:
strxor(c2, b2)

b'tly, the Customer sends\t\t\t\t\t\t\t\t\t'

# Decryption

In [200]:
Key

b'4103583911'

In [201]:
b1 = hash_256(Key + IV)
decrypted_p1 = strxor(b1, c1)
print(decrypted_p1)

b'Firstly, the Customer sends Firs'


In [202]:
decrypted_p1

b'Firstly, the Customer sends Firs'

In [203]:
b2 = hash_256(Key + c1)
decrypted_p2 = strxor(b2, c2)
print(decrypted_p2)

b'tly, the Customer sends\t\t\t\t\t\t\t\t\t'


In [205]:
decrypted_message = decrypted_p1 + decrypted_p2

In [206]:
decrypted_message.strip()

b'Firstly, the Customer sends Firstly, the Customer sends'